In [20]:
import pandas as pd
from pathlib import Path
from typing import List, Dict
from pydantic import BaseModel
from openai import OpenAI
import os
import json
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI client with API key from environment
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))



# Load the CSV file
df = "FullCallDataSummary.csv"

# Display the data as a table
df


'FullCallDataSummary.csv'

# Categorical Classification Function

In [16]:
# Define ClassificationResult model
class ClassificationResult(BaseModel):
    classification: str
    explanation: str

# Helper function for processing a single transcript
def _process_single_transcript(
    row,
    transcript_column: str,
    system_prompt: str,
    classifications_str: str,
    client
) -> ClassificationResult:
    """Process a single transcript and return classification result."""
    import json
    
    call_id = str(row.iloc[0])  # First column is call ID
    transcript_text = row[transcript_column]
    
    if pd.isna(transcript_text):
        # Handle missing transcript
        return ClassificationResult(
            explanation="No transcript text available for analysis",
            classification="None"
        )
    
    try:
        # Create the user prompt with the conversation
        user_prompt = f"""
        Call ID: {call_id}
        
        Transcript:-
        {transcript_text}
        
        Analyze this transcript and return the JSON with your classification and explanation.
        """
        
        # Make the API call using structured output
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            max_tokens=300,
            response_format={"type": "json_object"}
        )
        
        # Parse the response
        response_text = response.choices[0].message.content.strip()
        
        # Parse JSON and create ClassificationResult
        parsed_response = json.loads(response_text)
        result = ClassificationResult(
            explanation=parsed_response["explanation"],
            classification=parsed_response["classification"]
        )
        return result
            
    except Exception as e:
        print(f"Error processing call {call_id}: {str(e)}")
        return ClassificationResult(
            explanation=f"Error during processing: {str(e)}",
            classification="None"
        )

def classify_call_transcripts(
    classifications: List[str],
    dataframe: pd.DataFrame,
    context_prompt: str,
    transcript_column: str,
    classification_column: str = "Classification",
    explanation_column: str = "Explanation",
    max_workers: int = 8
) -> pd.DataFrame:
    """
    Classify call transcripts using LLM analysis with parallel processing.
    
    Args:
        classifications (List[str]): List of possible classifications (including "None")
        dataframe (pd.DataFrame): DataFrame with call data
        context_prompt (str): Context/question for classification
        transcript_column (str): Name of the column containing transcript text
        classification_column (str): Name for the output classification column
        explanation_column (str): Name for the output explanation column
        max_workers (int): Maximum number of parallel workers (default: 8)
    
    Returns:
        pd.DataFrame: Original DataFrame with added classification columns
    """
    from concurrent.futures import ThreadPoolExecutor
    import json
    
    # Initialize results list
    results: List[ClassificationResult] = []
    
    # Create classifications string for the prompt
    classifications_str = ", ".join(classifications)
    
    # Define the system prompt
    system_prompt = f"""
    You are an expert at analyzing sales call transcripts for classification purposes.
    
    Your task: {context_prompt}
    
    Available classifications: {classifications_str}
    
    For each conversation transcript, you must:
    1. Select exactly ONE classification from the provided list
    2. Provide a comprehensive explanation that includes:
        - Key customer statements or behaviors that influenced your decision
        - Any commitments, objections, or next steps mentioned
        - Your reasoning for why this classification best fits the conversation
    
    Return your response as a JSON object with this exact format:
{{
    "classification": "selected_classification_from_list", 
    "explanation": "One sentence explaining why you chose this classification"
}}
    
    Important: The classification must be exactly one of the provided options: {classifications_str}
    """
    
    # Process conversations in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        futures = [
            executor.submit(
                _process_single_transcript,
                row,
                transcript_column,
                system_prompt,
                classifications_str,
                client
            )
            for _, row in dataframe.iterrows()
        ]
        
        # Collect results as they complete
        for i, future in enumerate(futures):
            try:
                result = future.result()
                results.append(result)
                
                # Progress indicator (every 10 completions)
                if (i + 1) % 10 == 0:
                    print(f"Processed {i + 1}/{len(dataframe)} conversations...")
                    
            except Exception as e:
                print(f"Error in parallel processing: {str(e)}")
                # Add error result to maintain order
                results.append(ClassificationResult(
                    explanation=f"Error during parallel processing: {str(e)}",
                    classification="None"
                ))

    # Convert results list to DataFrame
    results_df = pd.DataFrame([result.model_dump() for result in results])
    
    # Merge with original dataframe using variable column names
    merged_df = dataframe.copy()
    merged_df[classification_column] = results_df['classification']
    merged_df[explanation_column] = results_df['explanation']
    
    return merged_df

Example Usage of Categorical Classification Function

In [ ]:
# Load the dataframe
dataframe = pd.read_csv(r"c:\Users\EricSheng\Downloads\FullCallDataSummary.csv", nrows=5)

# Define the classifications list
classifications = [
    "Business Secure Internet",
    "Business Fiber Internet", 
    "Business Secure Internet Plus",
    "Business Connection Backup",
    "PRO WiFi",
    "Business Phone",
    "Business Hosted Voice",
    "Business Trunking",
    "Business TV",
    "International/Specialty Channel Lineups",
    "Sports Packages",
    "Optimum Mobile for Business",
    "Premier Technical Support",
    "Service Plans/Device/IT Support Bundles",
    "None"
]

# Define the context prompt
context_prompt = "Identify which Optimum product or service is primarily discussed or mentioned in this sales call transcript. Consider both explicit mentions and implicit references to products/services."

#Run classify_call_transcripts
results = classify_call_transcripts(classifications, dataframe, context_prompt, "Conversation", "Product_Classification")
results

In [18]:
# Define the classifications list
classifications = [
    "Closed Sale",
    "Strong Lead",
    "Moderate Lead",
    "Weak Lead",
    "Not a Lead", 
    "Insufficient Data"
]

# Define the context prompt
context_prompt = """Analyze this sales call transcript and classify it into one of the following sales process stages based on the customer's level of engagement, interest, and the outcome of the conversation:

1. **Closed Sale** - Customer commits to purchasing/signing up during this call
2. **Strong Lead** - High intent, decision maker, budget available, urgent need/timeline
3. **Moderate Lead** - Moderate intent, needs identified, budget constraints or longer timeline
4. **Weak Lead** - Low intent, unclear needs, budget/authority concerns
5. **Not a Lead** - Customer explicitly declines, no budget/authority, or not viable
6. **Insufficient Data** - Call incomplete or customer interest unclear


Consider the following factors when classifying:
- Customer's explicit statements about interest level
- Whether any commitment or agreement was made
- Customer's tone and engagement level
- Specific next steps mentioned
- Whether the customer requested follow-up information or meetings
- Any objections raised and how they were addressed

Focus on the customer's behavior and statements rather than the salesperson's actions. A call should only be classified as "Definitive Sale on the Call" if there is clear evidence of a commitment or agreement made during this specific conversation."""

#Run classify_call_transcripts
results_with_sales_stage = classify_call_transcripts(classifications, dataframe, context_prompt, "Conversation", "stage")
results_with_sales_stage

Processed 10/10 conversations...


,NaturalId,Conversation,stage,Explanation
0,Call1,"[Agent] ""Thank you for choosing Optimum Busine...",Strong Lead,"The customer, Kimchi, expressed a clear need f..."
1,Call10,"[Agent] ""Thank you for choosing Optimum Busine...",Moderate Lead,The customer expressed interest in upgrading t...
2,Call100,"[Agent] ""Good morning. Thank you for calling O...",Moderate Lead,The customer expressed interest in upgrading t...
3,Call101,"[Agent] ""Good morning. Thank you for calling O...",Moderate Lead,The customer expressed interest in the service...
4,Call102,"[Agent] ""Hold on one second, hold on, do not d...",Closed Sale,"The customer, Rosa, provided detailed informat..."
5,Call103,"[Agent] ""Thank you for calling Optimum. No off...",Strong Lead,"The customer, Jay, shows high interest and eng..."
6,Call104,"[Agent] ""Thank you for calling Optimum Busines...",Closed Sale,The customer agreed to upgrade to the XGB fibe...
7,Call105,"[Agent] ""Thank you for calling O Business. Thi...",Strong Lead,"The customer, Ali Mohammed, expressed a clear ..."
8,Call106,"[Agent] ""Thank you for calling Optimum Busines...",Strong Lead,The customer expressed a high level of interes...
9,Call107,"[Agent] ""For calling Optimum for Business now ...",Closed Sale,The customer provided detailed information abo...


# Across-Unsupervized Grouping Function

In [ ]:
# Optimized Unsupervised Classification Function
from typing import Dict, List
from pydantic import BaseModel
import json

class Category(BaseModel):
    categoryName: str
    categoryDescription: str

class CondensingCategorization(BaseModel):
    condensed_categories: List[Category]
    category_mappings: Dict[str, List[str]]

def unsupervized_classification(
    dataframe: pd.DataFrame,
    input_column: str,
    context_prompt: str,
    id_column: str = None,
    target_column: str = "Group_Mapping"
) -> tuple[pd.DataFrame, CondensingCategorization]:
    """
    Unsupervised classification that groups text responses into unique categories.
    
    Args:
        dataframe: DataFrame with text data to categorize
        input_column: Name of the column containing text responses to categorize
        context_prompt: Context explaining what the text represents and grouping goal
        id_column: Name of the ID column (defaults to first column if None)
        target_column: Name for the output category column
    
    Returns:
        tuple: (DataFrame with added category column, LLM categorization output)
    """
    id_column = id_column or dataframe.columns[0]
    
    # Create input string for LLM
    input_data = "\n".join([f"ID: {str(row[id_column])} | {row[input_column]}" 
                           for _, row in dataframe.iterrows()])
    
    # System prompt
    system_prompt = f"""You are an expert at unsupervised text categorization and grouping.

Your task: {context_prompt}

You must:
1. Analyze all the provided text responses
2. Create 2-5 unique, meaningful categories that group similar responses
3. Provide clear category names and descriptions
4. Map each ID to exactly one category
5. Ensure every ID is assigned to exactly one category

Return your response as a JSON object with this exact format:
{{
    "condensed_categories": [
        {{
            "categoryName": "Category Name",
            "categoryDescription": "Description of what this category represents"
        }}
    ],
    "category_mappings": {{
        "Category Name": ["id1", "id2", "id3"],
        "Another Category": ["id4", "id5"]
    }}
}}"""

    # User prompt
    user_prompt = f"""Your goal is to categorize pieces of text into unique groups.

Context: {context_prompt}

Inputs to classify:
{input_data}

Return groupings that categorize each piece of text neatly into a single group."""

    print(f"Processing {len(dataframe)} text responses for unsupervised categorization...")
    
    try:
        # Make API call
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            max_tokens=1000,
            response_format={"type": "json_object"}
        )
        
        # Parse response
        parsed_response = json.loads(response.choices[0].message.content.strip())
        
        # Create output object
        llm_output = CondensingCategorization(
            condensed_categories=[
                Category(categoryName=cat["categoryName"], categoryDescription=cat["categoryDescription"])
                for cat in parsed_response["condensed_categories"]
            ],
            category_mappings=parsed_response["category_mappings"]
        )
        
        print(f"Created {len(llm_output.condensed_categories)} categories:")
        for cat in llm_output.condensed_categories:
            print(f"  - {cat.categoryName}: {cat.categoryDescription}")
        
        # Apply mappings to dataframe
        result_df = _apply_categorization(dataframe, llm_output, id_column, target_column)
        
        return result_df, llm_output
        
    except Exception as e:
        print(f"Error during unsupervised categorization: {str(e)}")
        return _create_error_result(dataframe, target_column)

def _apply_categorization(dataframe: pd.DataFrame, llm_output: CondensingCategorization, 
                         id_column: str, target_column: str) -> pd.DataFrame:
    """Apply categorization mappings to dataframe."""
    # Create ID to category mapping
    id_to_category = {}
    for category_name, id_list in llm_output.category_mappings.items():
        for id_val in id_list:
            id_to_category[str(id_val)] = category_name
    
    # Apply mapping
    result_df = dataframe.copy()
    result_df[target_column] = result_df[id_column].astype(str).map(id_to_category)
    
    # Handle unmapped items
    unmapped_count = result_df[target_column].isna().sum()
    if unmapped_count > 0:
        print(f"Warning: {unmapped_count} IDs could not be mapped to categories.")
        result_df = _handle_unmapped_items(result_df, llm_output, id_column, target_column, unmapped_count)
    
    return result_df

def _handle_unmapped_items(dataframe: pd.DataFrame, llm_output: CondensingCategorization,
                          id_column: str, target_column: str, unmapped_count: int) -> pd.DataFrame:
    """Handle unmapped items by assigning them to existing categories."""
    unmapped_rows = dataframe[dataframe[target_column].isna()]
    unmapped_data = "\n".join([f"ID: {str(row[id_column])} | {row[dataframe.columns[1]]}" 
                              for _, row in unmapped_rows.iterrows()])
    
    existing_categories = ", ".join([cat.categoryName for cat in llm_output.condensed_categories])
    
    unmapped_prompt = f"""You have {unmapped_count} items that need to be categorized into existing groups.

Existing categories: {existing_categories}

Unmapped items:
{unmapped_data}

Assign each unmapped item to the most appropriate existing category.
Return JSON: {{"unmapped_mappings": {{"id1": "Category Name", "id2": "Category Name"}}}}"""

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert at categorizing items into existing groups."},
                {"role": "user", "content": unmapped_prompt}
            ],
            temperature=0,
            max_tokens=500,
            response_format={"type": "json_object"}
        )
        
        unmapped_mappings = json.loads(response.choices[0].message.content.strip())["unmapped_mappings"]
        
        # Apply unmapped mappings
        for id_val, category_name in unmapped_mappings.items():
            dataframe.loc[dataframe[id_column].astype(str) == id_val, target_column] = category_name
            # Update category mappings
            if category_name in llm_output.category_mappings:
                llm_output.category_mappings[category_name].append(id_val)
            else:
                llm_output.category_mappings[category_name] = [id_val]
        
        print(f"Successfully categorized {unmapped_count} unmapped items.")
        
    except Exception as e:
        print(f"Error categorizing unmapped items: {str(e)}")
        dataframe[target_column] = dataframe[target_column].fillna("Unmapped")
    
    return dataframe

def _create_error_result(dataframe: pd.DataFrame, target_column: str) -> tuple[pd.DataFrame, CondensingCategorization]:
    """Create error result when categorization fails."""
    result_df = dataframe.copy()
    result_df[target_column] = "Error in categorization"
    
    empty_output = CondensingCategorization(
        condensed_categories=[],
        category_mappings={}
    )
    
    return result_df, empty_output


Example Usage of Unsupervized Grouping Function

In [53]:
# Example usage of unsupervized_classification function (unsupervised categorization)
# Using the output from classify_call_transcripts

# Define the context prompt for product categorization
context_prompt = "These are classifications of Optimum business products/services discussed in sales calls. We want to group these product classifications into 2-5 meaningful service categories. Determine the unique groups of these product classifications and name them appropriately. The goal is that all product classifications fit into one group exactly."

# Run unsupervised categorization on the results from classify_call_transcripts
categorized_results, categorization_output = unsupervized_classification(
    dataframe=results_cleaned,  # DataFrame from classify_call_transcripts
    input_column="Classification",  # Column containing the product classifications
    context_prompt=context_prompt,
    id_column="NaturalId",  # ID column from the original data
    target_column="Service_Group"  # Name for the new service group column
)

# Display the first 5 rows in a table format
print("Results with unsupervised product categorization (first 5 rows):")
display(categorized_results.head())

print("\nService categorization details:")
for category in categorization_output.condensed_categories:
    print(f"Service Group: {category.categoryName}")
    print(f"Description: {category.categoryDescription}")
    print(f"Product IDs in this group: {categorization_output.category_mappings[category.categoryName]}")
    print()


Processing 5 text responses for unsupervised categorization...
Unsupervised categorization complete!
Created 2 categories:
  - Communication Services: This category includes services related to communication such as phone and internet.
  - Entertainment Services: This category includes services related to entertainment such as TV.
Results with unsupervised product categorization (first 5 rows):


,NaturalId,Conversation,Classification,Explanation,Service_Group
0,0,"[Agent] ""Thank you for choosing Optimum Busine...",PRO WiFi,The conversation primarily revolves around set...,Entertainment Services
1,1,"[Agent] ""Thank you for choosing Optimum Busine...",Business Phone,The primary focus of the conversation is on up...,Communication Services
2,2,"[Agent] ""Good morning. Thank you for calling O...",Business Fiber Internet,The conversation primarily revolves around upg...,Communication Services
3,3,"[Agent] ""Good morning. Thank you for calling O...",Business Fiber Internet,The conversation primarily focuses on discussi...,Communication Services
4,4,"[Agent] ""Hold on one second, hold on, do not d...",Business TV,The conversation primarily revolves around set...,Entertainment Services



Service categorization details:
Service Group: Communication Services
Description: This category includes services related to communication such as phone and internet.
Product IDs in this group: ['1', '2', '3']

Service Group: Entertainment Services
Description: This category includes services related to entertainment such as TV.
Product IDs in this group: ['0', '4']



# Unique Value Splitter Function

In [21]:
def unique_value_splitter(
    dataframe: pd.DataFrame,
    splitter_column: str
) -> Dict[str, pd.DataFrame]:
    """
    Split a dataframe into separate dataframes based on unique values in a specified column.
    
    Args:
        dataframe (pd.DataFrame): DataFrame to split
        splitter_column (str): Name of the column to use for splitting
    
    Returns:
        Dict[str, pd.DataFrame]: Dictionary where keys are unique values and values are filtered DataFrames
    """
    
    # Get unique values from the splitter column
    unique_values = dataframe[splitter_column].unique()
    
    print(f"Splitting dataframe into {len(unique_values)} groups based on '{splitter_column}' column...")
    print(f"Unique values found: {list(unique_values)}")
    
    # Create dictionary to store split dataframes
    split_dataframes = {}
    
    # Split dataframe for each unique value
    for value in unique_values:
        # Filter dataframe for current value
        filtered_df = dataframe[dataframe[splitter_column] == value].copy()
        
        # Store in dictionary with value as key
        split_dataframes[str(value)] = filtered_df
        
        print(f"  - '{value}': {len(filtered_df)} rows")
    
    print("Dataframe splitting complete!")
    
    return split_dataframes

Example Usage of Unique Value Splitter Function

In [23]:
# Use unique_value_splitter on categorized_results
split_results = unique_value_splitter(results_with_sales_stage, "stage")

# Display the split results
print("\nSplit DataFrames:")
for group_name, group_df in split_results.items():
    print(f"\n{group_name} ({len(group_df)} rows):")
    display(group_df)


Splitting dataframe into 3 groups based on 'stage' column...
Unique values found: ['Strong Lead', 'Moderate Lead', 'Closed Sale']
  - 'Strong Lead': 4 rows
  - 'Moderate Lead': 3 rows
  - 'Closed Sale': 3 rows
Dataframe splitting complete!

Split DataFrames:

Strong Lead (4 rows):


,NaturalId,Conversation,stage,Explanation
0,Call1,"[Agent] ""Thank you for choosing Optimum Busine...",Strong Lead,"The customer, Kimchi, expressed a clear need f..."
5,Call103,"[Agent] ""Thank you for calling Optimum. No off...",Strong Lead,"The customer, Jay, shows high interest and eng..."
7,Call105,"[Agent] ""Thank you for calling O Business. Thi...",Strong Lead,"The customer, Ali Mohammed, expressed a clear ..."
8,Call106,"[Agent] ""Thank you for calling Optimum Busines...",Strong Lead,The customer expressed a high level of interes...



Moderate Lead (3 rows):


,NaturalId,Conversation,stage,Explanation
1,Call10,"[Agent] ""Thank you for choosing Optimum Busine...",Moderate Lead,The customer expressed interest in upgrading t...
2,Call100,"[Agent] ""Good morning. Thank you for calling O...",Moderate Lead,The customer expressed interest in upgrading t...
3,Call101,"[Agent] ""Good morning. Thank you for calling O...",Moderate Lead,The customer expressed interest in the service...



Closed Sale (3 rows):


,NaturalId,Conversation,stage,Explanation
4,Call102,"[Agent] ""Hold on one second, hold on, do not d...",Closed Sale,"The customer, Rosa, provided detailed informat..."
6,Call104,"[Agent] ""Thank you for calling Optimum Busines...",Closed Sale,The customer agreed to upgrade to the XGB fibe...
9,Call107,"[Agent] ""For calling Optimum for Business now ...",Closed Sale,The customer provided detailed information abo...


# Theme Analysis Function

## Helper Functions 
- `_generate_themes_for_batch()`: Generate themes for a small batch of transcripts (2 at a time)
    - Input: Batch of transcripts, column name, context prompt
    - Process: Sends transcripts to LLM, parse JSON response into Theme objects
    - Output: List of themes for that batch

- `_merge_themes_semantically()`: Merge similar themes from multiple batches into a final MECE set
    - Input: All themes from all batches, context prompt
    - Process: Send all themes to the LLM, asks it to merge similar ones while keeping MECE
    - Output: Final merged themes (3-10 themes)

In [ ]:
# _generate_themes_for_batch()

def _generate_themes_for_batch(batch_df: pd.DataFrame, transcript_column: str, context_prompt: str) -> List[Theme]:
    """Generate themes for a single batch of transcripts"""
    
    # Create transcript sample string with truncation
    def truncate_text(text, max_tokens=1500):
        # Rough estimate: 4 characters per token
        max_chars = max_tokens * 4
        if len(text) <= max_chars:
            return text
        return text[:max_chars] + "... [truncated]"
    
    transcript_sample = ""
    for idx, row in batch_df.iterrows():
        truncated_transcript = truncate_text(str(row[transcript_column]), 1500)
        transcript_sample += f"Transcript {idx}:\n{truncated_transcript}\n\n"
    
    # System prompt for theme generation
    system_prompt = f"""
    You are an expert at creating Mutually Exclusive and Collectively Exhaustive (MECE) categorization frameworks.
    
    Your task: {context_prompt}
    
    You must:
    1. Analyze the provided transcripts to identify 3-10 distinct themes
    2. Ensure themes are MECE:
       - Mutually Exclusive: No overlap between themes
       - Collectively Exhaustive: All relevant content fits into at least one theme
    3. Create clear, descriptive theme names and explanations
    4. Validate that your themes truly meet MECE criteria
    
    Return your response as a JSON object with this exact format:
    {{
        "themes": [
            {{
                "themeName": "Theme Name",
                "themeDescription": "Clear description of what this theme represents and how it differs from other themes"
            }}
        ],
        "mece_validation": "Explanation of how these themes are mutually exclusive and collectively exhaustive"
    }}
    """
    
    # User prompt for theme generation
    user_prompt = f"""
    Analyze the following transcripts and create a MECE framework of themes.
    
    Context: {context_prompt}
    
    Transcripts to analyze:
    {transcript_sample}
    
    Please generate 3-10 themes that are mutually exclusive and collectively exhaustive for this context.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            max_tokens=2000,
            response_format={"type": "json_object"}
        )
        
        response_text = response.choices[0].message.content.strip()
        import json
        parsed_response = json.loads(response_text)
        
        # Create Theme objects
        themes = [
            Theme(
                themeName=theme["themeName"],
                themeDescription=theme["themeDescription"]
            ) for theme in parsed_response["themes"]
        ]
        
        print(f"Batch theme generation complete: {len(themes)} themes created")
        return themes
        
    except Exception as e:
        print(f"Error during batch theme generation: {str(e)}")
        return []


In [ ]:
#_merge_themes_semantically()
def _merge_themes_semantically(all_themes: List[Theme], context_prompt: str) -> List[Theme]:
    """Merge similar themes using semantic analysis to create final MECE framework"""
    
    if len(all_themes) <= 1:
        return all_themes
    
    # Create theme list for LLM
    themes_text = "\n".join([f"- {theme.themeName}: {theme.themeDescription}" for theme in all_themes])
    
    # System prompt for theme merging
    system_prompt = f"""
    You are an expert at merging similar themes while maintaining MECE principles.
    
    Task: {context_prompt}
    
    You have been given themes from multiple batches. Merge similar themes while ensuring:
    1. Themes remain mutually exclusive (no overlap)
    2. Themes remain collectively exhaustive (all content fits into at least one theme)
    3. No information is lost in the merging process
    4. Create 3-10 final themes that best represent the data
    
    Return your response as a JSON object with this exact format:
    {{
        "themes": [
            {{
                "themeName": "Merged Theme Name",
                "themeDescription": "Clear description of what this merged theme represents"
            }}
        ],
        "mece_validation": "Explanation of how these merged themes are mutually exclusive and collectively exhaustive"
    }}
    """
    
    # User prompt for theme merging
    user_prompt = f"""
    Merge the following themes to create a final MECE framework:
    
    Original themes from batches:
    {themes_text}
    
    Please merge similar themes and create a final set of 3-10 themes that are mutually exclusive and collectively exhaustive.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            max_tokens=2000,
            response_format={"type": "json_object"}
        )
        
        response_text = response.choices[0].message.content.strip()
        import json
        parsed_response = json.loads(response_text)
        
        # Create merged Theme objects
        merged_themes = [
            Theme(
                themeName=theme["themeName"],
                themeDescription=theme["themeDescription"]
            ) for theme in parsed_response["themes"]
        ]
        
        print(f"Theme merging complete: {len(merged_themes)} final themes created from {len(all_themes)} original themes")
        return merged_themes
        
    except Exception as e:
        print(f"Error during theme merging: {str(e)}")
        # Return original themes if merging fails
        return all_themes

## Main Functions
- __*mece_theme_analysis()*__: Orchestrates the two-phase process
    - Flow:
        - Phase 1: Generate themes via __*_generate_mece_themes()*__
            1. Split transcripts into batches
            2. For each batch, call helper __*_generate_themes_for_batch()*__
            3. Collect all themes, and call __*_merge_themes_semantically()*__ to consolidate
            4. Return final themes
        - Phase 2: Classify transcripts via helper  __*_classify_transcripts_with_themes()*__
            1. Build a prompt with the final themes list
            2. Process transcripts in batches
            3. For each batch, send to LLM for classification
            4. Collect all classifications
            5. Return mappings + validation.

In [ ]:
# MECE Theme Analysis Function
from typing import Dict, List, Union
from pydantic import BaseModel

class Theme(BaseModel):
    themeName: str
    themeDescription: str

class MECEThemeAnalysis(BaseModel):
    themes: List[Theme]
    theme_mappings: Dict[str, List[str]]  # ID to list of theme names
    mece_validation: str  # Explanation of MECE compliance

def mece_theme_analysis(
    dataframe: pd.DataFrame,
    transcript_column: str,
    context_prompt: str,
    id_column: str = None, # default = first column if not specified
    target_column: str = "Theme_Analysis",
    themes_per_transcript: Union[int, str] = 1,  # 1 for single theme, "multiple" for multiple themes
    batch_size: int = 1
) -> tuple[pd.DataFrame, MECEThemeAnalysis]:
    """
    Two-phase MECE theme analysis: first generates themes, then classifies transcripts.
    
    Args:
        dataframe (pd.DataFrame): DataFrame with transcript data
        transcript_column (str): Name of the column containing transcript text
        context_prompt (str): Context/question for theme analysis
        id_column (str): Name of the ID column (defaults to first column if None)
        target_column (str): Name for the output theme column
        themes_per_transcript (Union[int, str]): Number of themes per transcript (1 or "multiple")
        batch_size (int): Number of items to process in each batch for classification phase
    
    Returns:
        tuple: (DataFrame with added theme column, MECE theme analysis output)
    """
    
    # Use first column as ID if not specified
    if id_column is None:
        id_column = dataframe.columns[0]
    
    print(f"Starting MECE theme analysis for {len(dataframe)} transcripts...")
    print(f"Context: {context_prompt}")
    
    # Phase 1: Generate MECE themes from all transcripts
    print("\n=== PHASE 1: Generating MECE Themes ===")
    themes = _generate_mece_themes(dataframe, transcript_column, context_prompt)
    
    if not themes:
        print("Error: No themes generated. Returning empty result.")
        result_df = dataframe.copy()
        result_df[target_column] = "Error: No themes generated"
        empty_output = MECEThemeAnalysis(
            themes=[],
            theme_mappings={},
            mece_validation="Error: No themes generated"
        )
        return result_df, empty_output
    
    print(f"Generated {len(themes)} themes:")
    for theme in themes:
        print(f"  - {theme.themeName}: {theme.themeDescription}")
    
    # Phase 2: Classify each transcript using the generated themes
    print(f"\n=== PHASE 2: Classifying Transcripts ===")
    theme_mappings, mece_validation = _classify_transcripts_with_themes(
        dataframe, transcript_column, themes, context_prompt, id_column, themes_per_transcript, batch_size
    )
    
    # Apply mappings to dataframe
    result_df = dataframe.copy()
    if themes_per_transcript == 1:
        # Single theme per transcript
        id_to_theme = {}
        for id_val, theme_list in theme_mappings.items():
            id_to_theme[str(id_val)] = theme_list[0] if theme_list else "Unclassified"
        result_df[target_column] = result_df[id_column].astype(str).map(id_to_theme)
    else:
        # Multiple themes per transcript
        id_to_themes = {}
        for id_val, theme_list in theme_mappings.items():
            id_to_themes[str(id_val)] = ", ".join(theme_list) if theme_list else "Unclassified"
        result_df[target_column] = result_df[id_column].astype(str).map(id_to_themes)
    
    # Handle unmapped items
    unmapped_count = result_df[target_column].isna().sum()
    if unmapped_count > 0:
        print(f"Warning: {unmapped_count} transcripts could not be mapped to themes.")
        result_df[target_column] = result_df[target_column].fillna("Unclassified")
    
    # Create final output
    final_output = MECEThemeAnalysis(
        themes=themes,
        theme_mappings=theme_mappings,
        mece_validation=mece_validation
    )
    
    print(f"\nMECE theme analysis complete!")
    print(f"Total themes: {len(themes)}")
    print(f"MECE validation: {mece_validation}")
    
    return result_df, final_output

def _generate_mece_themes(dataframe: pd.DataFrame, transcript_column: str, context_prompt: str) -> List[Theme]:
    """Phase 1: Generate MECE themes from all transcripts"""
    
    # Split transcripts into small batches for parallel processing
    batch_size = 1  # Process 2 transcripts per batch to avoid context limits
    batches = []
    for i in range(0, len(dataframe), batch_size):
        batch = dataframe.iloc[i:i+batch_size]
        batches.append(batch)

    print(f"Processing {len(dataframe)} transcripts in {len(batches)} batches of {batch_size}...")

    # Generate themes for each batch
    all_batch_themes = []
    for i, batch in enumerate(batches):
        print(f"Generating themes for batch {i+1}/{len(batches)}...")
        batch_themes = _generate_themes_for_batch(batch, transcript_column, context_prompt)
        if batch_themes:
            all_batch_themes.extend(batch_themes)

    if not all_batch_themes:
        print("Error: No themes generated from any batch.")
        return []

    print(f"Generated {len(all_batch_themes)} total themes across all batches.")

    # Print all themes before merging
    pd.reset_option('display.max_colwidth')
    print("\nAll themes generated before merging:")

    # First: Comma-separated list of theme names
    theme_names = [theme.themeName for theme in all_batch_themes]
    print("Themes:", ", ".join(theme_names))
    print()

    # Second: All themes with their explanations
    for i, theme in enumerate(all_batch_themes, 1):
        print(f"  {i}. {theme.themeName}: {theme.themeDescription}")
    print()

    # Merge similar themes using semantic analysis
    print("Merging similar themes...")
    merged_themes = _merge_themes_semantically(all_batch_themes, context_prompt)

    return merged_themes

def _classify_transcripts_with_themes(
    dataframe: pd.DataFrame, 
    transcript_column: str, 
    themes: List[Theme], 
    context_prompt: str, 
    id_column: str,
    themes_per_transcript: Union[int, str],
    batch_size: int
) -> tuple[Dict[str, List[str]], str]:
    """Phase 2: Classify each transcript using the generated themes"""
    
    # Create themes list for prompt
    themes_list = []
    for theme in themes:
        themes_list.append(f"- {theme.themeName}: {theme.themeDescription}")
    themes_text = "\n".join(themes_list)
    
    # Determine classification approach
    if themes_per_transcript == 1:
        classification_instruction = "Assign exactly ONE theme to each transcript."
        output_format = '{{"classifications": {{"id1": ["Theme Name"], "id2": ["Theme Name"]}}}}'
    else:
        classification_instruction = "Assign one or more relevant themes to each transcript."
        output_format = '{{"classifications": {{"id1": ["Theme 1", "Theme 2"], "id2": ["Theme 1"]}}}}'
    
    # System prompt for classification
    system_prompt = f"""
    You are an expert at classifying content using predefined themes.
    
    Your task: {context_prompt}
    
    Available themes:
    {themes_text}
    
    You must:
    1. {classification_instruction}
    2. Only use the provided themes (do not create new ones)
    3. Ensure each transcript is assigned to at least one theme
    4. Be consistent in your classification approach
    
    Return your response as a JSON object with this exact format:
    {output_format}
    """
    
    # Process in batches
    all_classifications = {}
    total_batches = (len(dataframe) + batch_size - 1) // batch_size
    print(f"Processing {len(dataframe)} transcripts in {total_batches} batches of {batch_size}...")
    
    for batch_num in range(total_batches):
        start_idx = batch_num * batch_size
        end_idx = min(start_idx + batch_size, len(dataframe))
        batch_df = dataframe.iloc[start_idx:end_idx]
        
        print(f"Processing batch {batch_num + 1}/{total_batches} (transcripts {start_idx}-{end_idx-1})...")
        
        # Create batch string
        batch_str = ""
        for idx, row in batch_df.iterrows():
            batch_str += f"ID: {str(row[id_column])}\nTranscript: {row[transcript_column]}\n\n"
        
        # User prompt for batch
        user_prompt = f"""
        Classify the following transcripts using the provided themes.
        
        Context: {context_prompt}
        
        Transcripts to classify:
        {batch_str}
        """
        
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0,
                max_tokens=1500,
                response_format={"type": "json_object"}
            )
            
            response_text = response.choices[0].message.content.strip()
            import json
            parsed_response = json.loads(response_text)
            
            # Store batch results
            all_classifications.update(parsed_response["classifications"])
            
            print(f"Batch {batch_num + 1} complete: {len(parsed_response['classifications'])} transcripts classified")
            
        except Exception as e:
            print(f"Error processing batch {batch_num + 1}: {str(e)}")
            continue
    
    # Generate MECE validation
    mece_validation = f"Generated {len(themes)} themes. Classification complete for {len(all_classifications)} transcripts."
    
    return all_classifications, mece_validation


Example Usage of Theme Analysis Function

In [ ]:
# Example usage of mece_theme_analysis
# Using the results from classify_call_transcripts

# Define the context prompt for sales technique analysis
context_prompt = "Analyze the sales techniques used by the sales agents in these call transcripts. Identify the different approaches, strategies, and methods used to engage customers and close sales. Focus on the behavioral patterns and communication styles of the sales agents."

# Run MECE theme analysis on the conversation transcripts
theme_results, theme_analysis = mece_theme_analysis(
    dataframe=results_cleaned,  # DataFrame from classify_call_transcripts
    transcript_column="Conversation",  # Column containing the conversation transcripts
    context_prompt=context_prompt,
    id_column="NaturalId",  # ID column from the original data
    target_column="Sales_Technique_Themes",  # Name for the new theme column
    themes_per_transcript=multiple,  # Single theme per transcript
    batch_size=1  # Process 2 transcripts at a time
)

# Display the first 5 rows in a table format
print("Results with MECE sales technique theme analysis (first 5 rows):")
display(theme_results.head())

print("\nGenerated themes:")
for theme in theme_analysis.themes:
    print(f"Theme: {theme.themeName}")
    print(f"Description: {theme.themeDescription}")
    print()

print(f"MECE Validation: {theme_analysis.mece_validation}")


# Usage of MECE Theme Analysis Function for Demo

In [24]:
# Usage of mece_theme_analysis
# Using the results from classify_call_transcripts

# Define the context prompt for sales technique analysis
context_prompt = (
    "Detect the product names being discussed for purchase, whether they are mentioned explicitly or implicitly by either the sales agent or the customer. If no formal product name is mentioned, use the conversation to determine the best fitting name for the product"
)

# Run MECE theme analysis on the conversation transcripts
theme_results, theme_analysis = mece_theme_analysis(
    dataframe=results_with_sales_stage,  # DataFrame from classify_call_transcripts
    transcript_column="Conversation",  # Column containing the conversation transcripts
    context_prompt=context_prompt,
    id_column="NaturalId",  # ID column from the original data
    target_column="Product being sold",  # Name for the new theme column
    themes_per_transcript=1,  # Single theme per transcript
    batch_size=1  # Process 2 transcripts at a time
)

# Display the first 5 rows in a table format
print("Results with MECE sales technique theme analysis (first 5 rows):")
display(theme_results)

print("\nGenerated themes:")
for theme in theme_analysis.themes:
    print(f"Theme: {theme.themeName}")
    print(f"Description: {theme.themeDescription}")
    print()

print(f"MECE Validation: {theme_analysis.mece_validation}")

Starting MECE theme analysis for 10 transcripts...
Context: Detect the product names being discussed for purchase, whether they are mentioned explicitly or implicitly by either the sales agent or the customer. If no formal product name is mentioned, use the conversation to determine the best fitting name for the product

=== PHASE 1: Generating MECE Themes ===
Processing 10 transcripts in 10 batches of 1...
Generating themes for batch 1/10...
Batch theme generation complete: 6 themes created
Generating themes for batch 2/10...
Batch theme generation complete: 6 themes created
Generating themes for batch 3/10...
Batch theme generation complete: 7 themes created
Generating themes for batch 4/10...
Batch theme generation complete: 7 themes created
Generating themes for batch 5/10...
Batch theme generation complete: 7 themes created
Generating themes for batch 6/10...
Batch theme generation complete: 6 themes created
Generating themes for batch 7/10...
Batch theme generation complete: 6 th

,NaturalId,Conversation,stage,Explanation,Product being sold
0,Call1,"[Agent] ""Thank you for choosing Optimum Busine...",Potential Interest as a Lead,The customer showed interest in the service an...,Internet Services
1,Call10,"[Agent] ""Thank you for choosing Optimum Busine...",Definitive Sale on the Call,"The customer, Heather, explicitly agreed to an...",Equipment and Installation
2,Call100,"[Agent] ""Good morning. Thank you for calling O...",Potential Interest as a Lead,The customer showed interest in porting their ...,Mobile Services
3,Call101,"[Agent] ""Good morning. Thank you for calling O...",Definitive Sale on the Call,The customer agreed to install the internet an...,Internet Services
4,Call102,"[Agent] ""Hold on one second, hold on, do not d...",Definitive Sale on the Call,"The customer, Rosa, explicitly committed to se...",Customer Needs and Usage
5,Call103,"[Agent] ""Thank you for calling Optimum. No off...",Definitive Sale on the Call,The customer committed to purchasing services ...,Mobile Services
6,Call104,"[Agent] ""Thank you for calling Optimum Busines...",Definitive Sale on the Call,The customer committed to upgrading their serv...,Internet Services
7,Call105,"[Agent] ""Thank you for calling O Business. Thi...",Potential Interest as a Lead,"The customer, Ali, shows interest in setting u...",Customer Needs and Usage
8,Call106,"[Agent] ""Thank you for calling Optimum Busines...",Definitive Sale on the Call,The customer explicitly committed to the servi...,Internet Services
9,Call107,"[Agent] ""For calling Optimum for Business now ...",Definitive Sale on the Call,The customer committed to the service by agree...,Internet Services



Generated themes:
Theme: Internet Services
Description: All discussions related to internet connectivity options, including types of internet (fiber, coax), speeds, pricing, installation details, and service reliability.

Theme: Mobile Services
Description: Any mention of mobile phone services, including plans, pricing, bundling with internet services, and features related to phone lines.

Theme: Equipment and Installation
Description: Covers the equipment provided for internet and mobile services, installation services, and discussions about the need for upgrades or new equipment.

Theme: Account Management
Description: Includes discussions about account setup, management, authorized users, billing preferences, and customer information collection.

Theme: Promotional Offers and Pricing
Description: Covers all promotional offers, discounts, pricing structures, and special deals available to customers, including contract terms and incentives for service upgrades.

Theme: Customer Needs